In [20]:
import requests as rq 
import json
from bs4 import BeautifulSoup as bs
import urllib3
import pandas as pd

In [2]:
urlhead = "https://www.drugbank.ca/unearth/q?"
frag1 = "utf8=%E2%9C%93&"
frag2 = "searcher=indications&"
query = "query="
filtersFrag= "&us=0&ca=0&eu=0&eu=1&commit=Apply+Filter"

In [3]:
params={'utf8':'%E2%9C%93' , 'searcher' : 'indications' , 'us':'0','ca':'0','eu':'1','commit':'Apply+Filter' , 'query':''}

In [4]:
# disease = "chronic+fatigue+syndrome"
params['query']="\"crohn's+disease\""
# url = urlhead + frag1 + frag2 + query + disease + filtersFrag
response = rq.get(urlhead,params=params)

In [5]:
soup = bs(response.text,'html.parser')
type(soup)

bs4.BeautifulSoup

In [6]:
synonym = soup.find_all('div', class_ = 'search-highlights')

In [7]:
drug_matches = soup.find_all('div', class_ = 'db-matches')

In [8]:
len(drug_matches) , len(synonym)

(6, 6)

In [9]:
# synonym

In [10]:
first = drug_matches[3]
first.text

u'Adalimumab'

In [11]:
MatchJson = {'Synonyms':[] , 'MatchedDrugs':[]}
for i in range(len(synonym)):
    syn = synonym[i]
    drug = drug_matches[i]
    synText = syn.text.replace(u'\u2026','')
    drugsName = drug.text.split('/')
    MatchJson['Synonyms'].append(synText)
    MatchJson['MatchedDrugs'].append(drugsName)

In [36]:
# MatchJson

In [14]:
DiseaseList = [ "bipolar", "\"chronic+fatigue+syndrome\"", "\"crohn's+disease\"" , 'dementia' , 'depression' ,'diabetes' , 'dysautonomia' , 
                'gastroparesis' , 'hypothyroidism' , "\"irritable+bowel+syndrome\"" , "\"interstitial+cystitis\"", "\"kidney+stones\"", "\"Meniere's disease\"" , "\"multiple+sclerosis\"" , 'parkinsons',
                'psoriasis','rheumatoid', "\"sleep+apnea\""]

In [15]:
urlhead = "https://www.drugbank.ca/unearth/q?"
frag1 = "utf8=%E2%9C%93&"
frag2 = "searcher=indications&"
query = "query="
filtersFrag= "&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter"
# params={'utf8':"%E2%9C%93" , 'searcher' : 'indications' , 'us':'0','ca':'0','eu':'0','eu':'1','commit':"Apply+Filter" , 'query':''}

DiseaseDrugJson = {}

for disease in DiseaseList:
    url = urlhead + frag1 + frag2 +query+disease+ filtersFrag
#     params['query']=disease
    response = rq.get(url)
    print response.url
    soup = bs(response.text,'html.parser')
    synonym = soup.find_all('div', class_ = 'search-highlights')
    drug_matches = soup.find_all('div', class_ = 'db-matches')
    print len(drug_matches) , len(synonym)
    MatchJson = {'Synonyms':[] , 'MatchedDrugs':[]}
    try:
        for i in range(len(synonym)):
            syn = synonym[i]
            drug = drug_matches[i]
            synText = syn.text.replace(u'\u2026','')
            drugsName = drug.text.split('/')
            MatchJson['Synonyms'].append(synText)
            MatchJson['MatchedDrugs'].append(drugsName)
    except:
        print "Something went horribly Wrong"
    DiseaseDrugJson[disease] = MatchJson

https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=bipolar&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
11 11
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22chronic+fatigue+syndrome%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
0 0
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22crohn's+disease%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
6 6
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=dementia&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
19 19
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=depression&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
25 25
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=diabetes&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
18 18
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=dysautonomia&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
0 0
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&

In [18]:
for k in DiseaseDrugJson:
    print k
    print len(DiseaseDrugJson[k]['Synonyms'])

psoriasis
25
rheumatoid
13
"multiple+sclerosis"
11
"sleep+apnea"
1
"interstitial+cystitis"
2
"irritable+bowel+syndrome"
5
parkinsons
1
dysautonomia
0
"kidney+stones"
1
"Meniere's disease"
1
bipolar
11
hypothyroidism
1
"crohn's+disease"
6
"chronic+fatigue+syndrome"
0
dementia
19
gastroparesis
2
depression
25
diabetes
18


In [19]:
# with open('DiseasesMatched.json','wb') as f:
#     json.dump(DiseaseDrugJson,f)

In [27]:
symptomList= pd.read_csv("../BL_Work/Dis_sym_edited.csv")

In [31]:
groups = symptomList.groupby('norm_UID')
symptoms = []
for name , group in groups:
    symptoms.append('\"' + name.strip().replace(' ','+') + '\"')
symptoms = list(set(symptoms))
print symptoms

['"Impairment+level+of+vision"', '"Backache"', '"Dyssomnia"', '"Loss+of+hair"', '"Mood+swings"', '"Tired"', '"Weight+gain"', '"Emotional+upset"', '"Disorientated"', '"Clouded+consciousness"', '"Stomach+ache"', '"Increased+thirst"', '"Sleep+deprivation"', '"Heartburn"', '"Abdominal+pain"', '"Swelling"', '"Feeling+agitated"', '"Impairment+of+balance"', '"Paresthesia"', '"Blurring+of+visual+image"', '"Feeling+angry"', '"Loss+of+appetite"', '"Pain"', '"Tremor"', '"Rectal+hemorrhage"', '"Abnormal+gait"', '"Impotence"', '"Eruption+of+skin"', '"Feeling+hopeless"', '"Indigestion"', '"Parkinson\'s+disease"', '"Trigeminal+neuralgia"', '"Bowel+dysfunction"', '"Parkinsonism"', '"Delusional+disorder"', '"Muscle+pain"', '"Pain+in+pelvis"', '"Malaise"', '"Thoughts+of+self+harm"', '"Dementia"', '"Depressive+disorder"', '"Hypothyroidism"', '"Reduced+mobility"', '"Jaundice"', '"Lack+of+energy"', '"Joint+pain"', '"Vertigo"', '"Headache"', '"Tinnitus"', '"Insomnia"', '"Multiple+sclerosis"', '"Dry+skin"', 

In [34]:
urlhead = "https://www.drugbank.ca/unearth/q?"
frag1 = "utf8=%E2%9C%93&"
frag2 = "searcher=indications&"
query = "query="
filtersFrag= "&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter"
# params={'utf8':"%E2%9C%93" , 'searcher' : 'indications' , 'us':'0','ca':'0','eu':'0','eu':'1','commit':"Apply+Filter" , 'query':''}

SymptomsDrugJson = {}

for disease in symptoms:
    url = urlhead + frag1 + frag2 +query+disease+ filtersFrag
#     params['query']=disease
    response = rq.get(url)
    print response.url
    soup = bs(response.text,'html.parser')
    synonym = soup.find_all('div', class_ = 'search-highlights')
    drug_matches = soup.find_all('div', class_ = 'db-matches')
    print len(drug_matches) , len(synonym)
    MatchJson = {'Synonyms':[] , 'MatchedDrugs':[]}
    try:
        for i in range(len(synonym)):
            syn = synonym[i]
            drug = drug_matches[i]
            synText = syn.text.replace(u'\u2026','')
            drugsName = drug.text.split('/')
            MatchJson['Synonyms'].append(synText)
            MatchJson['MatchedDrugs'].append(drugsName)
    except:
        print "Something went horribly Wrong"
    SymptomsDrugJson[disease] = MatchJson

https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Impairment+level+of+vision%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
0 0
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Backache%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
25 25
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Dyssomnia%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
0 0
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Loss+of+hair%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
0 0
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Mood+swings%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
0 0
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Tired%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
3 3
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Weight+gain%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
1 1
https://www.drugbank.ca/une

https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Swollen+abdomen%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
1 1
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Loss+of+interest%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
0 0
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Forgetful%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
0 0
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Pins+and+needles%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
0 0
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Irritable+bowel+syndrome%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
5 5
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Weight+loss%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
3 3
https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&searcher=indications&query=%22Vomiting%22&us=1&ca=0&eu=0&eu=1&commit=Apply+Filter
25 25
https://ww

In [35]:
for k in SymptomsDrugJson:
    print k
    print len(SymptomsDrugJson[k]['Synonyms'])

"Impairment+level+of+vision"
0
"Backache"
25
"Dyssomnia"
0
"Lack+of+energy"
1
"Mood+swings"
0
"Tired"
3
"Weight+gain"
1
"Emotional+upset"
0
"Disorientated"
0
"Clouded+consciousness"
0
"Stomach+ache"
0
"Increased+thirst"
0
"Sleep+deprivation"
0
"Heartburn"
25
"Abdominal+pain"
1
"Swelling"
3
"Feeling+agitated"
1
"Impairment+of+balance"
0
"Paresthesia"
0
"Blurring+of+visual+image"
0
"Feeling+angry"
0
"Loss+of+appetite"
1
"Pain"
25
"Impaired+cognition"
0
"Tremor"
2
"Rectal+hemorrhage"
0
"Abnormal+gait"
0
"Impotence"
1
"Eruption+of+skin"
0
"Feeling+hopeless"
0
"Indigestion"
3
"Parkinson's+disease"
7
"Trigeminal+neuralgia"
1
"Bowel+dysfunction"
1
"Parkinsonism"
7
"Delusional+disorder"
0
"Muscle+pain"
25
"Restlessness"
4
"Malaise"
0
"Increased+appetite"
0
"Dementia"
19
"Depressive+disorder"
12
"Hypothyroidism"
1
"Reduced+mobility"
1
"Jaundice"
1
"Loss+of+hair"
0
"Joint+pain"
25
"Vertigo"
2
"Crying+associated+with+mood"
0
"Tinnitus"
0
"Insomnia"
3
"Multiple+sclerosis"
11
"Dry+skin"
21
"Blood+i

In [37]:
# with open('SymptomsMatched.json','wb') as f:
#     json.dump(SymptomsDrugJson,f)

In [39]:
def cleanDictionary(diseaseList , DiseaseMatch):
    for k in diseaseList:
        for i in range(len(DiseaseMatch[k]['Synonyms'])):
            findString = k.replace('+',' ').lower()
            sourceString = DiseaseMatch[k]['Synonyms'][i].lower()
            if sourceString.find(findString) < 0:
                print "found pointless synonym"
                print DiseaseMatch[k]['Synonyms'][i]

In [44]:
# symptomList

In [42]:
cleanDictionary(symptoms,SymptomsDrugJson)

found pointless synonym
Matched Synonyms:  Simple backache  Matched Title:  Simple backache  
found pointless synonym
Matched Synonyms:  Backache ... Backache NOS ... Backache, unspecified  Matched Drugs products:  Aspirin Backache ... Extra Strength Aspirin Backache ... Super Extra Strength Aspirin Backache  
found pointless synonym
Matched Synonyms:  Backache ... Backache NOS ... Backache, unspecified  Matched Title:  Backache  
found pointless synonym
Matched Synonyms:  Backache ... Backache NOS ... Backache, unspecified  Matched Title:  Backache, unspecified  
found pointless synonym
Matched Drugs products:  Aspirin Backache ... Extra Strength Aspirin Backache ... Super Extra Strength Aspirin Backache  
found pointless synonym
Matched Drugs products:  Aspirin Backache ... Extra Strength Aspirin Backache ... Super Extra Strength Aspirin Backache  
found pointless synonym
Matched Drugs products:  Aspirin Backache ... Extra Strength Aspirin Backache ... Super Extra Strength Aspirin Ba

In [43]:
SymptomsDrugJson

{'"Abdominal+bloating"': {'MatchedDrugs': [[u'Magaldrate']],
  'Synonyms': [u'Matched Synonyms:  Abdominal bloating  ']},
 '"Abdominal+pain"': {'MatchedDrugs': [[u'Hyoscyamine']],
  'Synonyms': [u'Matched Synonyms:  Abdominal Pain ... Abdominal pain NOS  ']},
 '"Abnormal+gait"': {'MatchedDrugs': [], 'Synonyms': []},
 '"Abnormal+sexual+function"': {'MatchedDrugs': [], 'Synonyms': []},
 '"Amnesia"': {'MatchedDrugs': [], 'Synonyms': []},
 '"Anger+reaction"': {'MatchedDrugs': [], 'Synonyms': []},
 '"Anxiety"': {'MatchedDrugs': [[u'Bromazepam ',
    u' Chlordiazepoxide ',
    u' Clorazepic acid ',
    u' Hydroxyzine'],
   [u'Buspirone ',
    u' Chlordiazepoxide ',
    u' Diazepam ',
    u' Lorazepam ',
    u' Meprobamate ',
    u' Oxazepam'],
   [u'Chlorpromazine'],
   [u'Acetylsalicylic acid ',
    u' Alprazolam ',
    u' Amoxapine ',
    u' Chlordiazepoxide ',
    u' Diazepam ',
    u' Doxepin ',
    u' Lorazepam ',
    u' Maprotiline ',
    u' Methotrimeprazine ',
    u' Oxazepam ',
    

In [45]:
with open('SymptomsMatched.json','wb') as f:
    json.dump(SymptomsDrugJson,f)